In [ ]:
# импортируем необходимые библиотеки
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep


In [ ]:
# полученные cookies и headers с сайта и сконвертированные для языка Python
cookies = {
    'esadovod-optuShopErrMsg': '',
    'esadovod-optuShopSID': '52K38uaaiP',
    'ucvid': '7gQoQ0v6yp',
    'itemViewMode': 'grid',
    '_ym_uid': '1679501044958866511',
    '_ym_d': '1679501044',
    '_ym_isad': '1',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'ru,en;q=0.9',
    'Connection': 'keep-alive',
    # 'Cookie': 'esadovod-optuShopErrMsg=; esadovod-optuShopSID=52K38uaaiP; ucvid=7gQoQ0v6yp; itemViewMode=grid; _ym_uid=1679501044958866511; _ym_d=1679501044; _ym_isad=1',
    'Referer': 'https://sadovod-opt.com/shop/all',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.5.708 Yowser/2.5 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Yandex";v="23"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}
# r являеться запросом к странице
r = requests.get('https://sadovod-opt.com/shop/zhenskaja-odezhda;2', cookies=cookies, headers=headers)

In [ ]:
# Показывает что мы получили
r.text

In [ ]:
# переменная библиотеки BeautifulSoup для приведения полученной информации в надлежащий вид
soup = BeautifulSoup(r.text, 'lxml')

In [ ]:
# Получение названия товара
name = soup.find('div', class_='list-item').find('a', class_='item-name').text
name

'Костюм женский'

In [ ]:
# Получение ссылки на товар
link = 'https://sadovod-opt.com' + soup.find('div', class_='list-item').find('a', class_='item-name').get('href')
link

'https://sadovod-opt.com/shop/1780442/desc/kostjum-zhenskij'

In [ ]:
# Получение даты размещения
date_of_placement = soup.find('div', class_='list-item').find('div', class_='item-info').find('p').text.split()[1]
date_of_placement

'22.03.2023'

In [ ]:
# Получение Артикула
art = soup.find('div', class_='list-item').find('div', class_='item-info').find('p').text.split()[3]
art

't917518'

In [ ]:
# Получение Кода товара
key = soup.find('div', class_='list-item').find('div', class_='item-info').find('p').text.split()[-1]
key

'1780657'

In [ ]:
# Получение цены товара
price = soup.find('div', class_='list-item').find('div', class_='item-info').find('div', class_='item-price').find('span').text
price

'1037.00 '

In [ ]:
# Получение описанния товара
description = soup.find('div', class_='list-item').find('div', class_='item-info').find('div', class_='item-text').text
description

'При отсутствии выбранной расцветки возможна замена. Размер: 1.5 спальный Пододеяльник: 150*220 (1 шт) Простыня: 150*220 (1 шт) Наволочка: 70*70 (2 шт)'

In [ ]:
items = soup.findAll('div', class_='list-item')
len(items)

50

In [ ]:
# Создаём цикл с запросами к серверу сайта Садовод для получения всей нужной нам информации
data = []
# Цикл для пролистывания страниц на сайте 
for p in range(1, 110):
  print(p)
  url = f'https://sadovod-opt.com/shop/zhenskaja-odezhda;{p}'
  r = requests.get(url, cookies=cookies, headers=headers)
  sleep(3) # Таймер для того что-бы не перегружать сервер
  soup = BeautifulSoup(r.text, 'lxml')
  

  items = soup.findAll('div', class_='list-item')
# Цикл для получения информации с сайта
  for item in items:
      name = item.find('a', class_='item-name').text
      link = 'https://sadovod-opt.com' + item.find('a', class_='item-name').get('href')
      try: # Эта модель для того чтоб наш парсер работал даже при условиях невозможности получения информации с какой либо страницы
        date_of_placement = item.find('div', class_='item-info').find('p').text.split()[1]
      except IndexError: # Второе условие для модели try 
        date_of_placement = 'NaN'
      try:
        art = item.find('div', class_='item-info').find('p').text.split()[3]
      except IndexError:
        art = 'NaN'
      try:
        key = item.find('div', class_='item-info').find('p').text.split()[-1]
      except IndexError:
        key = 'NaN'
      price = item.find('div', class_='item-info').find('div', class_='item-price').find('span').text
      try:
        description = item.find('div', class_='item-info').find('div', class_='item-text').text
      except IndexError:
        description = 'NaN'

      data.append([name, link, date_of_placement, art, key, price, description])
# Хочу заметить что модель try-except добавил практически в каждую строку (хотя в этом не было сильной нужды) для отладки всего парсинга


In [ ]:
# Создадим переменную для колонок нашего датасета
columns = ['Название', 'Ссылка', 'Дата размещения', 'Артикул', 'Код', 'Цена', 'Описание']

In [ ]:
# Создаём датасет и записываем его в csv файл для дальнейшей работы
df = pd.DataFrame(data, columns=columns)
df.to_csv('Список женской одежды с Садовода.csv')

In [ ]:
# Выведем хвост датасета , что бы увидеть количество получившихся данных
df.tail() 

,Название,Ссылка,Дата размещения,Артикул,Код,Цена,Описание
5395,Трусы женские бесшовные,https://sadovod-opt.com/shop/1766258/desc/trus...,13.03.2023,242978,1766258,183.00,"Картинка Бабочка светоотражающая, Бесшовный Со..."
5396,Трусы женские бесшовные,https://sadovod-opt.com/shop/1766257/desc/trus...,13.03.2023,242978,1766257,183.00,"Картинка Бабочка светоотражающая, Бесшовный Со..."
5397,Трусы женские бесшовные,https://sadovod-opt.com/shop/1766256/desc/trus...,13.03.2023,242978,1766256,183.00,"Картинка Бабочка светоотражающая, Бесшовный Со..."
5398,Трусы женские бесшовные,https://sadovod-opt.com/shop/1766255/desc/trus...,13.03.2023,242978,1766255,183.00,"Картинка Бабочка светоотражающая, Бесшовный Со..."
5399,Трусы женские бесшовные,https://sadovod-opt.com/shop/1766254/desc/trus...,13.03.2023,242978,1766254,183.00,"Картинка Бабочка светоотражающая, Бесшовный Со..."
